# MPI

MPI (Message Passing Interface) is a standard for writing parallel programs that can run on a distributed memory system. It is widely used in the HPC (High Performance Computing) community. There are implementations of MPI for many programming languages, including C, C++, Fortran, and Python. Much of the functionality and many of the commands we will see here will be similar in other languages. 

There are multiple implementations of MPI, including OpenMPI, MPICH, and Intel MPI. If you are running this notebook in a Github Codespace, MPICH will already be installed. If you want to install MPICH on your local system you can follow the instructions [here](https://www.mpich.org/downloads/).

We will also be using the `mpi4py` package which is a Python wrapper for MPI. This is already installed if you'r running this notebook in a GitHUb Codespace, or you can install it locally using pip:

```bash
pip install mpi4py
```

Unlike the other methods we have seen so far, we run MPI from the terminal using the `mpiexec` command. We can run a Python script named `python_script.py` using MPI like this:

```bash
mpiexec -n 4 python python_script.py
```
In this command, the `-n 4` flag tells MPI to run the script using 4 processes. The text `python python_script.py` tells which command MPI should be running on each process. This will run the script 4 times in parallel.

## Ranks

Each copy of the code will be executed on its own process and will be identified by its "rank". The rank is a unique integer identifier for each process which can be accessed using the `mpi4py.MPI.COMM_WORLD.Get_rank()` function. The code below shows how to access the rank:

```python
import mpi4py.MPI as MPI

# Get a reference to the current MPI.COMM_WORLD communicator
comm = MPI.COMM_WORLD

# Get the rank of the current process
rank = comm.Get_rank()

# Print the rank of the current process
print(f'This script is being run by Rank {rank}')
```

This code can be found in the file `04_mpi_scripts/get_rank.py`. To run this command we will need to change directory in the terminal using the command:

```bash
cd 04_mpi_scripts
```

and run the script using MPI using the command:

```bash
mpiexec -n 4 python 04_mpi_scripts/get_rank.py
```